# OpenStreetMap Case Study: Santiago
    

## Map Area

Santiago, Chile

<ul>
<li> https://www.openstreetmap.org/#map=12/-33.4568/-70.5882  </li>
<li> https://mapzen.com/data/metro-extracts/metro/santiago_chile/ </li>
</ul>

As an expat living in Santiago, I am always trying to familiarize myself more with where I live.  I'm interested in how complete this data is and in what I can find out by querying.  Maybe I'll even improve my Spanish a little bit. 

Initial inspection of the downloaded osm file in the terminal reveals that it is about 260 MB large.  I used the code provided in the project details (see footnote 1) to create a small sample to work with.  From now on I will use this sample to audit and work with the data until otherwise stated.

## Sample creation and first look at data

In [12]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "santiago.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))
    output.write('</osm>')

In [13]:
#prints out the first 10 elements in the sample osm

for i, element in enumerate(get_element('sample.osm')):
    print(ET.tostring(element, encoding='utf-8'))
    if i == 10:
        break

<node changeset="41984301" id="386131" lat="-33.4415548" lon="-70.6423656" timestamp="2016-09-07T17:17:35Z" uid="4339485" user="DoodleBob" version="4" />
	
<node changeset="41967734" id="386225" lat="-33.4345622" lon="-70.6600699" timestamp="2016-09-06T22:03:46Z" uid="4339485" user="DoodleBob" version="24">
		<tag k="highway" v="traffic_signals" />
	</node>
	
<node changeset="41986638" id="386259" lat="-33.4403085" lon="-70.6453085" timestamp="2016-09-07T18:47:05Z" uid="4339485" user="DoodleBob" version="12">
		<tag k="highway" v="traffic_signals" />
	</node>
	
<node changeset="42989269" id="386287" lat="-33.4391923" lon="-70.6581065" timestamp="2016-10-18T17:15:44Z" uid="3778777" user="Baconcrisp" version="9">
		<tag k="highway" v="traffic_signals" />
		<tag k="traffic_signals" v="signal" />
	</node>
	
<node changeset="41989874" id="386306" lat="-33.4331618" lon="-70.6546245" timestamp="2016-09-07T21:16:44Z" uid="4339485" user="DoodleBob" version="12">
		<tag k="highway" v="traffic_si

These elements look pretty ordinary: the first ten are nodes, created by a few users (including President George W. Bush!)  Let's keep exploring.  In the future I would like to see how many users contributed various percentages of the data.

## Data Exploration

In [14]:
#Prints the type of tags and number of tags 

import pprint

def count_tags(filename):
    tag_dict = {}
    iter = ET.iterparse(open(filename, "r"))
    for _, item in iter:
        if item.tag in tag_dict.keys():
            tag_dict[item.tag] +=1
        else:
            tag_dict[item.tag] = 1
    return tag_dict

tags = count_tags('sample.osm')
pprint.pprint(tags)
    

{'member': 1367,
 'nd': 116336,
 'node': 96124,
 'osm': 1,
 'relation': 198,
 'tag': 116569,
 'way': 23636}


In our sample of the data, we find the tags that we expected, nodes ways, and relations, as well as secondary tags, nds, which are nodes insides of ways, and members which are elements inside of relations.  Nothing out of the ordinary.

In [15]:
# Checks to find out a bit more about the tags and their format 
# I.e. are the tags all in lower case? Do they include colons or problematic characters?

import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        attr = element.attrib['k']
        if lower.search(attr):
            keys["lower"] += 1
        elif lower_colon.search(attr):
            keys["lower_colon"] += 1
        elif problemchars.search(attr):
            keys["problemchars"] +=1
        else:
            keys["other"] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

keys = process_map('sample.osm')
pprint.pprint(keys)

{'lower': 58039, 'lower_colon': 58311, 'other': 216, 'problemchars': 3}


The vast majority of the tags are lower case or are lower case with a colon. Only 3 tags have problematic characters or are in the other category (in this sample).  Let's check those out.

In [16]:
#Prints out tags with problematic characters as well as their 'v' values

def probchars(filename):
    prob_tags = {}
    for _, element in ET.iterparse(filename):
        if element.tag == "tag":
            attr = element.attrib['k']
            if problemchars.search(attr):
                prob_tags[element] = attr, element.attrib['v']
    return prob_tags

print "Problematic tags:"
pprint.pprint(probchars('sample.osm'))


Problematic tags:
{<Element 'tag' at 0x9189f28>: (u'Reconocimiento cartogr\xe1fico de campo 2016 por KG.',
                                'Kg Ground Survey 2016'),
 <Element 'tag' at 0x15858128>: (u'Reconocimiento cartogr\xe1fico de campo 2016 por KG.',
                                 'Kg Ground Survey 2016'),
 <Element 'tag' at 0x1f0aaef0>: ('largo fachada',
                                 '40 m por Rosales / 10 m por Dieciocho')}


These keys "Reconocimiento cartogr\xe1fico de campo 2016 por KG." and 'largo fachada'included an accent sign though nothing in largo fachada ("long facade") needs it.  Clearly, they can still be processed and most accented characters are covered by UTC-8.  **Update: Later "Reconocimiento ... " was found to be a common source of data.  This key is clearly mislabed, it should be the value for a the key tag "source".**

In [17]:
# Returns a dictionary of keys used in tags, and how many times they appear

def tagkeys(filename):
    tag_keys = {}
    for _, element in ET.iterparse(filename):
        if element.tag == "tag":
            attr = element.attrib['k']
            if attr in tag_keys.keys():
                tag_keys[attr] +=1
            else:
                tag_keys[attr] =1
    return tag_keys

tag_key_values = tagkeys('sample.osm')
print "Tag Keys:"
pprint.pprint(tag_key_values)

Tag Keys:
{'FIXME': 1,
 u'Reconocimiento cartogr\xe1fico de campo 2016 por KG.': 2,
 'Source': 174,
 'access': 350,
 'addr:city': 947,
 'addr:country': 370,
 'addr:district': 1,
 'addr:door': 1,
 'addr:full': 78,
 'addr:housename': 66,
 'addr:housenumber': 21573,
 'addr:interpolation': 10118,
 'addr:postcode': 38,
 'addr:state': 32,
 'addr:street': 22855,
 'admin_level': 25,
 'aerialway': 1,
 'aeroway': 26,
 'alt_name': 21,
 'am': 1,
 'amenity': 1542,
 'amenity_1': 1,
 'antenna': 2,
 'area': 38,
 'atm': 25,
 'attribution': 1164,
 'backrest': 2,
 'barrier': 1193,
 'bench': 4,
 'bicimapa_acceso': 10,
 'bicimapa_ambiente': 10,
 'bicimapa_calle': 10,
 'bicimapa_capacidad': 10,
 'bicimapa_comentario': 6,
 'bicimapa_comuna': 10,
 'bicimapa_direccion': 10,
 'bicimapa_edificio': 9,
 'bicimapa_fecha': 10,
 'bicimapa_horarios': 10,
 'bicimapa_latitud': 10,
 'bicimapa_longitud': 10,
 'bicimapa_nombre': 10,
 'bicimapa_numero': 9,
 'bicimapa_ocupacion': 10,
 'bicimapa_precio': 10,
 'bicimapa_precis

In [18]:
#Shows tag keys based on the frequency
#adapted from footnote3

d_view = [ (v,k) for k,v in tag_key_values.iteritems() ]
d_view.sort(reverse=True) 
for v,k in d_view:
    print "%s: %d" % (k,v)

addr:street: 22855
addr:housenumber: 21573
name: 10176
addr:interpolation: 10118
highway: 9979
source: 3871
id_origin: 3722
surface: 3274
building: 2907
oneway: 1951
amenity: 1542
ref: 1303
barrier: 1193
attribution: 1164
shop: 1085
ts_orientacion: 1071
ts_hacia: 1071
ts_calle: 1071
route_ref: 1071
ref_name: 1071
is_in:city: 1071
ts_codigo: 1069
ts_desde: 1065
addr:city: 947
natural: 851
lanes: 828
landuse: 645
leisure: 458
noexit: 373
addr:country: 370
access: 350
building:levels: 323
operator: 297
power: 287
maxspeed: 283
school:type: 230
contact:email: 219
type: 199
layer: 196
golf: 176
Source: 174
sport: 158
service: 142
crossing: 138
bridge: 130
restriction: 126
office: 118
wheelchair: 105
cuisine: 101
railway: 90
website: 83
addr:full: 78
religion: 77
phone: 70
traffic_signals: 69
chile:region: 68
is_in: 67
emergency: 66
addr:housename: 66
bicycle: 64
waterway: 60
foot: 60
tunnel: 51
tourism: 51
fixme: 50
contact:phone: 50
capacity: 49
place: 45
public_transport: 44
noname: 44
ma

## Frequent Secondary Tags

The most frequent tags are "addr:street:" (22,855) and "addr:housenumber:" (21,573). First I'll validate that "addr:housenumber" are in fact only numbers, then I will explore the values for the streets a bit more.  The fourth most frequent tag is "addr:interpolation" which is used to indicate a list or range of addresses.  This is a good way of distinguishing from house numbers with a dash in them, a not infrequent practice here.  Potentially though, some of the house numbers with dashes could mean ranges of house numbers and some of them could indicate a correct house number.  An external data source would be necesary to validate this.

In [19]:
#Check to see that all house numbers are actually integers, return a list of non-int values, 

def try_int(s):
    try:
        return int(s)
    except ValueError:
        return str(s)

def house_numbers(filename):
    number_errors = []
    for _, element in ET.iterparse(filename):
        if element.tag == "tag":
            if element.attrib['k'] == "addr:housenumber":
                if not isinstance( try_int(element.attrib['v']), int ):
                    number_errors.append(element.attrib['v'])
    return number_errors

pprint.pprint(house_numbers('sample.osm'))
print len(house_numbers('sample.osm'))


['3087-A',
 '21;3520',
 '419;748',
 '225;40',
 '740;750',
 '2060;2080',
 '142;215',
 '21;043',
 '21;0173',
 '9672;9687',
 '2;10631',
 '6892;6891',
 '6998;6999',
 '343;344;6828;6826;6825;6827',
 '23;21',
 '02-2498750',
 '157 A',
 's/n',
 '3101;1929',
 '504 B',
 '5413 Local 550',
 '5081 Local 12',
 '2843 b',
 'S/N',
 '346, Local F3',
 '012?',
 '5413 Local 417',
 '21-187',
 '21;1714',
 '267 B',
 '287 E',
 '314 E',
 '106 A',
 '133 A',
 '162 A',
 '153 E',
 '1191;2435',
 'municipalidad santiago - ministerio desarrollo social',
 '264G',
 '11-39',
 '233-219',
 '386 A',
 '7261 G']
43


Though there are some really unusual values (such as a place name and lists of numbers, which should be listed as interpolations) many of the non-int house numbers included a letter, which is fairly common here.  Additionally, some addresses are marked as "S/n" which means "sin numero" (without a number) which is also not uncommon here. It does not make sense to change these since they reflect the reality of addresses here.  Let's move on to street names.

In [20]:
#Update addr:housenumber values which do not include a number in them

numbers = re.compile('\d')
                     
def is_housenumber(element):
    return (element.attrib['k'] == "addr:housenumber") 

# if a housenumber does not contain any numeric characters, it's key tag is changed to a name                     
def addressfix(element):
    if is_housenumber(element):
        if not(numbers.search(element.attrib['v'])):
            if (element.attrib['v'] != "s/n"): 
                if (element.attrib['v'] != "S/N"):
                    print element.attrib['v']
                    element.attrib['k'] = 'name'
    return element


In [21]:
#Test to see if correct element is found

for _, element in ET.iterparse('sample.osm'):
    if element.tag == "tag":
        element = addressfix(element)

municipalidad santiago - ministerio desarrollo social


## Street names ("addr:street")

In [22]:
# Returns a list of street names and how many times they appear

def streets(filename):
    streets = {}
    for _, element in ET.iterparse(filename):
        if element.tag == "tag":
            if element.attrib['k'] == "addr:street":
                if element.attrib['v'] not in streets:
                    streets[element.attrib['v']] = 1
                else:
                    streets[element.attrib['v']] += 1
    return streets

street_values = streets('sample.osm')
print "Streets:"
pprint.pprint(street_values)

Streets:
{'1 Nor Oriente': 1,
 '1 Norte': 5,
 '1 Oriente': 8,
 '1 Poniente': 2,
 '1 Sur': 7,
 '1 de Mayo': 2,
 '11 De Octubre': 1,
 '11 Oriente': 3,
 '11 de Enero': 1,
 '11 de Julio': 1,
 '12 Norte': 1,
 '12 de Febrero': 1,
 '13 Norte': 3,
 '13 Oriente': 1,
 '14 De Octubre': 2,
 '14 Norte': 3,
 '14 Oriente': 1,
 '14 de Julio': 1,
 '14 de la Fama': 3,
 '15 Norte': 3,
 '16 Norte': 2,
 '16 Poniente': 1,
 '17 Norte': 3,
 '18 De Septiembre': 2,
 '18 Norte': 5,
 '18 de Septiembre': 9,
 '19 De Septiembre': 1,
 '19 Norte': 2,
 '1ro de Mayo': 5,
 '2 Norte': 1,
 '2 Oriente': 4,
 '2 Poniente': 3,
 '2 Sur': 1,
 '20 Norte': 2,
 '21 De Enero': 1,
 '21 de Mayo': 6,
 '23 de Febrero': 6,
 '26 de Enero': 1,
 '3 Nor Oriente': 2,
 '3 Norte': 4,
 '3 Oriente': 2,
 '3 Palos': 2,
 '3 Puntas': 2,
 '3 de Diciembre': 2,
 '30 de Octubre': 1,
 '4 Norte': 2,
 '4 Oriente': 5,
 '4 Poniente': 4,
 '4 Remos': 1,
 '4 Vientos': 1,
 '5 Norte': 4,
 '5 Oriente': 1,
 '5 Sur Poniente': 1,
 '5 de Abril': 2,
 '5 de Mayo': 1,
 '6

This list reflects that in Chile, most street names are *just* a name -- that is to say they don't have a classifier  such as "Calle" or "Avenida" after them like many western countries have a "Street", "Avenue", "Rue", "Strasse" or some other kind of street name after the actual proper noun.  Additionally, if such a classifier exists, it is at the front of phrase, rather than at the end.  (I.e.  "Avenida Presidente Kennedy" rather than "President Kennedy Ave.") Still, scanning quickly through it, it was possible to see that some abbreviations were being made (Av. or Avda. for Avenida.)  To look for other abbreviations, I decided to look at street names with periods.

In [23]:
#Prints street names with "."

streetcount = 0
for street in street_values:
    if street.find('.')!=-1:
        streetcount +=1
        print street

print "There are {0} streets with the '.' character in them.".format(streetcount)

Psje. Lago Bolsena
Co. Laguna Azul
Pje. Francisco Coloane
Pje. Ricardo Boizard
Avenida Central Pob.La Sel
A. Fernandez
JOSE M.CARO
Gran Avenida Parad.29
Av. Jose Maria Caro
H. Ortiz
Avda. Brasil
Pje. Las Alpacas
Av. Echenique
Co. Negro
Av. Eliodoro Yanez
Psje. Co. HermitaÃ±o
Psje. Co. Bravo
Co. Chicharras
Av. Lo Ovalle
Psje. Manuel San Martin
Psje. Co. Chena
Av. Presidente Kennedy
Pje. El Canelo
Av. Cerrillos
Sta. Francisca
Pje. El Roble
Avda. Valdivia
Psje. Victor Donoso
Pje. Los linces
Comandante B. SepÃºlveda
Cap. Layseca
Pje. Los Pelicanos
Ciclovia Central Card. Silva Henríquez
Av. Presidente Eduardo Frei Montalva
Avda. Camilo Henríquez
Avda. El Rodeo
Psje. 32
Psje. 31
Psje. 30
Psje. 35
Jose M. Velasco
Avda. Italia
Psje. Co. La Parva
Avda. Blanco Encalada / Exposicion
Psje. 19
Psje. 18
Psje. 12
Psje. 14
Ciclovia Central Card. Silva Henriquez
Avda. Las Condes
Av. Rodrigo de Araya
Sta. Gabriela
Av. Ossa
Psje. Gabriel Slgado
Psje. El Gabino
Av. Lo Barnechea
Av. Padre Hurtado
Av. Liber

The most common abbreviations where Av. and variants on Avenida, Psje. and Pje and variants on Pasaje and Co., which after some research I found to be "Cerro."  (Yay for learning new things!)  Additionally there were a couple of special cases, "Sta." for "Santa" and "Fco" for Francisco. **This is another change that should be implemented.**  For now let's check out what kinds of things are going into the third most common attribute "name."

## "Name" elements

In [24]:
# Returns a list of values for the tag key "name" and how many times they appear

def namefinder(filename):
    names = {}
    for _, element in ET.iterparse(filename):
        if element.tag == "tag":
            if element.attrib['k'] == "name":
                if element.attrib['v'] not in names:
                    names[element.attrib['v']] = 1
                else:
                    names[element.attrib['v']] += 1
    return names

name_values = namefinder('sample.osm')
pprint.pprint(name_values)

{'': 5,
 ' ': 1,
 u'"El Aut\xe9ntico" Pollo a las Brasas': 1,
 '(M) El Golf': 1,
 '(M) Franklin': 1,
 '(M) H. de Magallanes': 1,
 '(M) La Cisterna': 1,
 u'(M) Pr\xedncipe de Gales': 1,
 u'(M) San Ram\xf3n': 1,
 '(M) Toesca': 1,
 '(M) Trinidad': 1,
 '1': 1,
 '1 Esperanza': 1,
 '1 Norte': 1,
 '1 Oriente': 1,
 '1 Poniente': 1,
 '1 Sur': 2,
 '1 de Enero': 1,
 '1 y 2 Juzgado de Letras': 1,
 '10 al 15 Juzgado de Garantia': 1,
 '11 de Octubre': 2,
 '11 de Septiembre': 1,
 '12 Norte': 1,
 '12 de Febrero': 1,
 '13 de Enero': 1,
 u'13\xb0 Comisaria La Granja': 1,
 u'14 de La Fama / Bajos de Jim\xe9nez': 1,
 '14 de Octubre / Santa Teresita': 1,
 '15 Norte': 1,
 u'16 Norte / Domingo Santa Mar\xeda': 1,
 '17 Norte': 1,
 '17 Sur': 1,
 '18 De Septiembre': 1,
 '18 de Septiembre': 1,
 u'18\xb0 Comisaria \xd1u\xf1oa': 1,
 '19 Norte': 1,
 '19 Sur / Del Ferrocarril': 1,
 '19 de Septiembre / Felix Margoz': 1,
 u'19\xb0 CB Santiago': 1,
 u'1ra Compa\xf1\xeda de Bomberos de Maip\xfa': 1,
 '2 Norte': 1,
 '2 O

Scrolling through the data, it looks as though there are similar clean-ups to be made as in the street:addr tag. (Avenida and other street names being normalized.)  This makes me wonder if some of the items in this category should be put be values for the "addr:street" key rather than "name" key.  Then again, it is possible that a user made a node that contained both of these secondary attributes.  **Still, if the value is a street namne, they key should be "addr:street" not "name".**

Another thing that stuck at me was that though there are only 1542 amenities, there are 10,176 names.  Scanning through the list made it seem that far more than 15% of them were amenities.  It is possible that some nodes have "name" tags but not the appropriate amenity tags.  **Further exploration should be done to see if a node was tagged with both a addr:street and name tag, or amenity without a name tag to see if mislabeling has taken place.**

The last bit of exploration I will do is to look at the elements tagged addr:interpolation, highway, and source.  The next most common tag is "id_origin" which is not found on the OSM documentation, so I'll have to check that out too, out of curiousity's sake.

In [25]:
streets = ["Av.", "Ave", "Avda.", "Avenida", "Calle", "Camino", "Diagonal",  "Pje", "Pje.", "Psje", "Pasaje"]

def name_street_fix(element):
    if element.attrib['k'] == "name":
        # Checks to see if the name value matches an already given street name, this is particularly useful for the streets
        # which are dates.  The tag key is changed to "addr:street"
        for street in street_values:
            if element.attrib['v'] == street:
                    element.attrib['k'] = 'addr:street'
        # If the name value has a common street classifier in it, the tag key is changed to "addr:street"   
        for street in streets:
            if element.attrib['v'].find(street):
                element.attrib['k'] = 'addr:street'
    return element
            

In [26]:
#I've already created this method for street names, and "names" so I decided to create a more generalized version

def tagfinder(filename, tag):
    names = {}
    for _, element in ET.iterparse(filename):
        if element.tag == "tag":
            if element.attrib['k'] == tag:
                if element.attrib['v'] not in names:
                    names[element.attrib['v']] = 1
                else:
                    names[element.attrib['v']] += 1
  
    return names

tagfinder('sample.osm', 'addr:interpolation')

{'Las Hualtatas': 1, 'even': 5054, 'odd': 5063}

The majority of addr:interpolation tags are used to indicate whether the houseing numbers are even or odd, a legitimate use of this tag according to the documentation. The odd man out "Las Hualtatas" is clearly a mistake and should have an addr:street tag. (It's really a very nice street in a swanky part of town.)  **This is an easy one time fix.**

In [27]:
#Creates the fix (changing the key to addr:street, rather than addr:interpolation when the value is not even or odd) 
# to be used later when shaping elements

def Hualtatasfix(element):
    if element.attrib['k'] == 'addr:interpolation':
        if (element.attrib['v'] != 'even' and element.attrib['v'] != 'odd'):
            element.attrib['k'] = 'addr:street'
    return element
            
        

## Highways

In [28]:
#Prints highway tag values
tagfinder('sample.osm', 'highway')

#Method to print a dictionary ordered by its values
def dict_print_by_value(d):
    d_view = [ (v,k) for k,v in d.iteritems() ]
    d_view.sort(reverse=True) 
    for v,k in d_view:
        print "%s: %d" % (k,v)

dict_print_by_value(tagfinder('sample.osm', 'highway'))

residential: 4590
bus_stop: 1087
living_street: 1053
service: 549
footway: 453
traffic_signals: 409
tertiary: 394
secondary: 301
primary: 258
crossing: 178
track: 139
motorway_link: 93
motorway: 80
primary_link: 54
path: 54
pedestrian: 44
steps: 40
turning_circle: 36
unclassified: 34
secondary_link: 30
construction: 25
tertiary_link: 22
cycleway: 15
proposed: 10
motorway_junction: 8
give_way: 8
stop: 4
road: 4
mini_roundabout: 3
bridleway: 2
raceway: 1
escape: 1


For the large part, it seems as if the elements with the highway tag have been properly used.  "Residential" and "living_street" are appropriately used according the the documentation.  The "bus_stop" tags may or may not be accurate.  According to the documentation: *"highway=bus_stop should be used for "A small bus stop. Can be mapped more rigorously using public_transport=stop_position for the position where the vehicle stops and public_transport=platform for the place where passengers wait. See public_transport= for more details."*  See footenote 4. Without further knowledge about whether these are "small bus stops" or more major ones, this change cannot be made. This could be verified by looking at data from other sources, such as http://www.transantiago.cl.  Speaking of sources...

## Data sources

In [29]:
dict_print_by_value(tagfinder('sample.osm', 'source'))

http://www.transantiago.cl: 1070
Reconocimiento cartográfico 2016 por KG.: 655
Bing: 538
Reconocimiento cartográfico de campo 2016 por KG: 365
Reconocimiento cartográfico de campo 2016 por KG.: 305
Gobierno Regional Metropolitano de Santiago (http://www.gobiernosantiago.cl/): 216
Yahoo: 170
KG Ground Survey 2016: 169
Instituto Nacional de Estadisticas (INE) http://www.ine.cl/: 122
vialidad.cl: 63
Instituto Nacional de Estadisticas (www.ine.cl): 53
Reconocimiento cartográfico por KG 2016: 25
GNS: 12
instituto nacional de estadisticas www.ine.cl: 11
Instituto Nacional de estaditicas www.ine.cl: 10
Bicimapa - UAHC: 10
Bing 2012: 8
survey: 6
Instituto Nacional de Estaditicas www.ine.cl: 4
Instituto Nacional De Estadisticas: 4
BingMap: 4
yahoo: 3
http://sitmaipu.maipu.cl/flexviewers/AVerdes/: 3
JAXA/ALOS/AVNIR-2, 2010-02-27: 3
www.tarjetabip.cl: 2
www.junji.gob.cl: 2
li: 2
datos.gob.cl antenas.subtel.gob.cl: 2
GPS: 2
www.baik.cl: 1
visit: 1
ourairports.com: 1
https://www.servipag.com/: 1
ht

The source tag is very useful for validating the accuracy of the data.  As can be seen here, a large amount of the source data is from https://www.transantiago.cl/, the public transportation agency for the city.  Scraping data from this website would be the best way to verify the validity of the data.  The next major source of data is "Reconocimiento cartográfico 2016 por KG" or "Reconocimiento cartográfico de camp 2016 por KG" this is an unknown data source to me, and googling it does not reveal much. (Only links back to openstreetmaps.) However, there are many redundant tags for this and "instituto nacional de estadisticas" which should be consolidated.  **This is another plausible update.**  

The other larger contributors of data, Bing, and Gobierno Regional Metropolitano de Santiago (http://www.gobiernosantiago.cl/) are well consolidated.  It would be interesting to see if the multiple users used these same sources or if data from one source was only provided by one or two users.

In [30]:
#mapping dictionary to update sources
sources= {'www.ine.cl' : 'Instituto Nacional de Estadistica www.ine.cl',
            'Instituto Nacional De Estadisticas': 'Instituto Nacional de Estadistica www.ine.cl',
            'Bing' : "Bing",
          "bing" : "Bing",
          "2016 por KG" : "Reconocimiento cartográfico 2016 por KG"}
          
#updates source names
def sourcefix(element):
    if element.attrib['k'] == 'source':
        for key in sources.keys():
            if element.attrib['v'].find(key) > -1:
                element.attrib['v'] = sources[key]
    return element
                            

## "id_origin"

In [31]:
dict_print_by_value(tagfinder('sample.osm', 'id_origin'))

90505: 2
87464: 2
86661: 2
86502: 2
86365: 2
85641: 2
84126: 2
83800: 2
833: 2
83150: 2
82232: 2
8074: 2
79809: 2
79307: 2
78981: 2
78713: 2
78440: 2
77260: 2
75640: 2
74398: 2
71747: 2
71215: 2
70440: 2
70338: 2
69377: 2
68975: 2
68504: 2
6574: 2
64174: 2
63242: 2
61397: 2
60482: 2
60481: 2
60430: 2
59257: 2
59155: 2
58576: 2
58495: 2
57985: 2
57137: 2
56467: 2
55400: 2
53504: 2
5283: 2
48209: 2
45887: 2
45716: 2
4527: 2
44846: 2
4417: 2
43340: 2
42959: 2
42110: 2
420: 2
41151: 2
40121: 2
39637: 2
39069: 2
3834: 2
34841: 2
34208: 2
29796: 2
25936: 2
24379: 2
222640: 2
220445: 2
219762: 2
219392: 2
218226: 2
215165: 2
19307: 2
189023: 2
188839: 2
188791: 2
187327: 2
186619: 2
185904: 2
185038: 2
184270: 2
183233: 2
182143: 2
181678: 2
179744: 2
179402: 2
178328: 2
177305: 2
176893: 2
1765: 2
174434: 2
173135: 2
172332: 2
171470: 2
170279: 2
166960: 2
166958: 2
165092: 2
165020: 2
164971: 2
164001: 2
161471: 2
160591: 2
160282: 2
159954: 2
158933: 2
157319: 2
157117: 2
156179: 2
155843:

This is a very mysterious tag indeed, it is not to be found in the OSM documentation. A possible theory is that these are user id numbers, then again, perhaps this is a tag used by only one user. Furtherwork needs to be done to discover the meaning of this common but unclear tag.

# Recommendations for changes and additional exploration

By exploring the data, some necessary changes or causes for further investigation became clear:

<ul>
<li>"Addr:street" and "name" names need to be standardized, though this is not as big of an issue in western countries, since many street names here are only a proper noun. "Avda." should become "Avenida", etc.</li>
<li>House numbers that contain ranges should be changed to the "addr:interpolation" tag.  House numbers that contain a strings (e.g. "municipalidad santiago - ministerio desarrollo social") should be changed to an amenities or street names.</li>
<li>Improperly tagged "addr:interpolation" ("Las Hualtatas e.g.") should be changed to a "addr:street" </li>
<li>The use of "name" tags should be verified.  Are they being used for the same purpose as "street_addr:" tags? Are amenities being correctly named?</li>
<li>Data sources should be made uniform, especially for the large common data sources which have several variants of the name. </li>
<li>More work should be done to discover the meaning of the "id_origin" tag.</li>
</ul>


In [32]:
#Creates a dictionary of the secondary tags found in a node

tree = ET.parse('sample.osm')
root = tree.getroot()
nodecount = 0
node_dict = {}
for node in root.findall('node'):
    node_dict[nodecount] = {}
    tagcount = 0
    for tag in node.findall('tag'):
        node_dict[nodecount][tagcount] = tag.attrib['k'] + ":" + (tag.attrib['v'])
        tagcount +=1
    nodecount += 1


In [33]:
#Prints tags by node, if the node does not contain an empty dictionary
for i in node_dict:
    if bool(node_dict[i]):
        pprint.pprint(node_dict[i])

{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals', 1: 'traffic_signals:signal'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals', 1: 'traffic_signals:signal'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals', 1: 'traffic_signals:signal'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals', 1: 'traffic_signals:signal'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:traffic_signals'}
{0: 'highway:tra

In [40]:
#Creates a dictionary of nodes, each node has a list of the secondary tags assigned to it
tag_list={}
nodecount=0
for node in root.findall('node'):
    tag_list[nodecount] = []
    for tag in node.findall('tag'):
        tag_list[nodecount].append(tag.attrib['k'])
    nodecount += 1


In [41]:
#Counts the number of nodes which have both an amenity and name tag and nodes that have only an amenity tag with no name
#Same for street names and regular name tags

ns_count =0
na_count = 0
amenity_only = 0
street_only =0

for node in tag_list:
    if ('name' in tag_list[node]):
        if ('amenity' in tag_list[node]):
            na_count +=1
        if ('addr:street' in tag_list[node]):
            ns_count +=1
    elif ('amenity' in tag_list[node]):
        amenity_only +=1
    elif ('addr:street' in tag_list[node]):
        street_only +=1
        
print "Nodes with both amenity and name tags: " + str(na_count)
print "Nodes with with an amenity but no name tag: " + str(amenity_only)
print
print "Nodes with both addr:street and name tags: " + str(ns_count)
print "Nodes with with street but no name tag: " + str(street_only)

Nodes with both amenity and name tags: 909
Nodes with with an amenity but no name tag: 136

Nodes with both addr:street and name tags: 1043
Nodes with with street but no name tag: 20500


Only about 13% of amenities are nameless.  Noticeably the sample has 1542 amenities, but only 1045 listed here. Presumably the rest are not nodes, but rather ways or relations. (A football field may have several nodes to demarcate the edges and thus would be a way.)  Still, it seems that the majority of amenities are named.  On the otherhand, most of the nodes labeled with a street:addr do not additionally have a name though some do.  Perhaps these 1043 are duplicately named.

In [42]:
#The function and dictionary for fixing abbreviations in street names
from collections import defaultdict

mapping = { "A.": "Avenida",
            "Av.": "Avenida",
            "Av" : "Avenida",
            "Avda." : "Avenida",
            "Avda" : "Avenida",
            "Ave.": "Avenida",
            "Ave": "Avenida",
            "Co.": "Cerro",
            "Co" : "Cerro",
            "Psje." : "Pasaje",
            "Psje" : "Pasaje",
            "Pje." : "Pasaje",
            "Pje" : "Pasaje",
            "Fco." : "Francisco",
            "Fco" : "Francisco",
            "Sta. " : "Santa",
            "Sta" : "Santa"
            }

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street" or elem.attrib['k'] == "name" )

def update_streetname(name, mapping):
    for error in mapping.keys():
        if error in name:
            name = re.sub(r'\b' + error + r'\b\.?', mapping[error], name)
    return name


## CSV File creation

In [43]:

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}



In [44]:
# Runs all local fixes on an element

def fix(element):
    element = addressfix(element)
    element = name_street_fix(element)
    element = Hualtatasfix(element)
    element = sourcefix(element)
    if is_street_name(element):
        element.attrib['v'] = update_streetname(element.attrib['v'], mapping)
    return element

In [45]:
#Creates csv files from the osm file

import cerberus
import codecs
import csv


OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    way_tags = []
    
    #Create node data structure
    if element.tag == 'node':
    #Fills in node_attribs dictionary
        for item in element.attrib:
            if item in node_attr_fields:
                node_attribs[item] = element.attrib[item]
    #Fills in secondary tags list of dicts
        for child in element:
            temp = {}
            temp["id"] = element.attrib['id']
            temp["value"] = child.attrib['v']
            child = fix(child)
            if re.match(problem_chars, child.attrib['k']):
                pass
            k = child.attrib['k'].split(":")
            if len(k)==1:
                temp["type"] = default_tag_type
                temp["key"] = child.attrib["k"]
            if len(k)==2:
                temp["type"] = k[0]
                temp["key"] = k[1]
            elif len(k)>2:
                k = child.attrib['k'].split(":", 1)
                temp["type"] = k[0]
                temp["key"] = k[1]
            tags.append(temp)

    #Create way data structure
    if element.tag == 'way':
    #Fills in the way attribs dictionary
        for item in element.attrib:
            if item in way_attr_fields:
                way_attribs[item] = element.attrib[item]
                
    #Fills in the way_nodes list
        count = 0
        for child in element:
            if child.tag == "nd":
                temp1 = {}
                temp1["id"] = element.attrib["id"]
                temp1["node_id"] = child.attrib["ref"]
                temp1["position"] = count
                count +=1
                way_nodes.append(temp1)
                
    #Fills in the tags list of dicts for way
            if child.tag == "tag":
                temp2 = {}
                temp2["id"] = element.attrib["id"]
                temp2["value"] = child.attrib["v"]
                if re.match(problem_chars, child.attrib['k']):
                    pass
                child =fix(child)
                k = child.attrib['k'].split(":")
                if len(k)==1:
                    temp2["type"] = default_tag_type
                    temp2["key"] = child.attrib["k"]
                if len(k)==2:
                    temp2["type"] = k[0]
                    temp2["key"] = k[1]
                elif len(k)>2:
                    k = child.attrib['k'].split(":", 1)
                    temp2["type"] = k[0]
                    temp2["key"] = k[1]
                way_tags.append(temp2)
                
    if element.tag == 'node':            
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': way_tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)


municipalidad santiago - ministerio desarrollo social


# Data Overview

### File Sizes

After the creation of the CSV files, I created an SQLite Database and imported each CSV as a table in the database.  

From there, I decided to do the rest of the queries programatically, rather than in the command terminal.

### Number of nodes and ways

In [46]:
import sqlite3

# Fetch records from santiago.db
db = sqlite3.connect("santiago.db")
c = db.cursor()

#Count the number of nodes
QUERY = '''
SELECT COUNT(*)
FROM nodes;
'''
c.execute(QUERY)
rows = c.fetchall()
print rows[0][0]

961239


In [47]:
#Count the number of ways
QUERY = '''
SELECT COUNT(*)
FROM ways;
'''
c.execute(QUERY)
rows = c.fetchall()
print rows[0][0]

236355


### Number of unique users

In [48]:
QUERY = '''
SELECT COUNT(DISTINCT(uid))          
FROM (SELECT uid FROM nodes 
UNION SELECT uid FROM ways);
'''
c.execute(QUERY)
rows = c.fetchall()

print rows[0][0]


1489


Let's compare this to the result of code which looks at the osm file directly.

In [49]:
#Finds the number of unique users contributing to the data set

import xml.etree.ElementTree as ET

def get_user(element):
    if element.get('uid'):
        return element.get('uid')
        print element.get('uid')
    else:
        return None


def number_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if get_user(element):
            if get_user(element) not in users:
                users.add(get_user(element))
    return users

users = number_users('santiago.osm')
print "There are {0} unique users contributing to this data set.".format(len(users))

There are 1493 unique users contributing to this data set.


Very interesting indeed -- though close, these numbers are not exactly the same.  Perhaps users were lost when written to the CSV files?  Perhaps there are four users who only contributes relations and not ways or nodes?  The former seems much more likely and begs the question of whether other data was dropped when converting formats.

### User statistics

In [50]:
#Finds the top ten contributing users
QUERY = '''
SELECT nodes_ways.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) nodes_ways
GROUP BY nodes_ways.user
ORDER BY num DESC
LIMIT 10; '''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'Julio_Costa_Zambelli', 206514),
 (u'Fede Borgnia', 196366),
 (u'felipeedwards', 95448),
 (u'chesergio', 59706),
 (u'dintrans_g', 56384),
 (u'madek', 32527),
 (u'Baconcrisp', 31644),
 (u'toniello', 26331),
 (u'Chilestreet', 25054),
 (u'Run_cl', 22982)]


In [51]:
#Finds the top five amenity tags from the top user 'Julio_Costa_Zambelli'
QUERY = '''
SELECT tags.value, COUNT(*) as count 
FROM (SELECT key, user, value FROM (nodes JOIN nodes_tags ON nodes.id=nodes_tags.id) UNION ALL 
SELECT key, user, value FROM (ways JOIN ways_tags ON ways.id=ways_tags.id))tags

WHERE tags.user =  'Julio_Costa_Zambelli'
and tags.key = 'amenity'
GROUP BY tags.value
ORDER BY count DESC
limit 5; '''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'school', 938),
 (u'kindergarten', 124),
 (u'parking', 121),
 (u'bank', 74),
 (u'pharmacy', 74)]


In [52]:
#Prints the number of users contributing once
QUERY = '''
SELECT COUNT(*) 
FROM
    (SELECT e.user, COUNT(*) as num
     FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
     GROUP BY e.user
     HAVING num=1)  u;'''

c.execute(QUERY)
rows = c.fetchall()

print rows[0][0]

439


In [53]:
#Prints the number of users contributing more than 10,000 elements
QUERY = '''
SELECT COUNT(*) 
FROM
    (SELECT e.user, COUNT(*) as num
     FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
     GROUP BY e.user
     HAVING num>10000)  u;'''

c.execute(QUERY)
rows = c.fetchall()

print rows[0][0]

20


In [55]:
QUERY = '''
SELECT sum(u.num) 
FROM
    (SELECT e.user, COUNT(*) as num
     FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
     GROUP BY e.user
     HAVING num>10000)  u;'''
c.execute(QUERY)
rows = c.fetchall()

print rows[0][0]     

916783


In [56]:
QUERY = '''
SELECT sum(u.num) 
FROM
    (SELECT e.user, COUNT(*) as num
     FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
     GROUP BY e.user
)  u;'''
c.execute(QUERY)
rows = c.fetchall()

print rows[0][0]     

1197594


### Distinct Comunas

In [54]:
#Finds the number of entries under various tags with the "city" key

QUERY = '''
SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) tags
WHERE tags.key LIKE '%city'
GROUP BY tags.value
ORDER BY count DESC; '''

c.execute(QUERY)
rows = c.fetchall()


pprint.pprint(rows)

[(u'Providencia', 3415),
 (u'Santiago', 1620),
 (u'Las Condes', 1591),
 (u'La Reina', 963),
 (u'Puente Alto', 896),
 (u'La Florida', 826),
 (u'San Bernardo', 812),
 (u'Maip\xfa', 798),
 (u'\xd1u\xf1oa', 733),
 (u'Pudahuel', 538),
 (u'La Cisterna', 473),
 (u'Macul', 447),
 (u'La Pintana', 410),
 (u'Lo Barnechea', 377),
 (u'Quinta Normal', 372),
 (u'Pe\xf1alol\xe9n', 356),
 (u'Recoleta', 355),
 (u'Vitacura', 354),
 (u'Quilicura', 343),
 (u'El Bosque', 316),
 (u'La Granja', 314),
 (u'Estaci\xf3n Central', 310),
 (u'Cerrillos', 305),
 (u'Conchal\xed', 305),
 (u'Lo Prado', 282),
 (u'Lo Espejo', 277),
 (u'Cerro Navia', 269),
 (u'Renca', 255),
 (u'Huechuraba', 238),
 (u'San Joaqu\xedn', 234),
 (u'San Miguel', 219),
 (u'San Ram\xf3n', 199),
 (u'Pedro Aguirre Cerda', 197),
 (u'Independencia', 171),
 (u'Buin', 106),
 (u'Las Condes, METROPOLITANA', 73),
 (u'Padre Hurtado', 68),
 (u'Maipu', 50),
 (u'Pe\xf1alolen', 50),
 (u'6', 44),
 (u'Colina', 44),
 (u'Estacion Central', 44),
 (u'15', 41),
 (u'20

Not surprisingly, the highest number of "city" tags falls into Providencia, which is one of the most popular areas of the city.  (And where I live!) Runners-up are other comunas with high numbers of restaurants, schools, residential areas, etc.  We can see that this is another area for improvement since there are some redundancies, e.g. the comuna of Providencia is represented in 6 different ways: 'Providencia', 'Providencia,Santiago', 'Santiago;Providencia', 'Providencia;Santiago', 'Providencia;Santiago', 'Providencias'.  Additionally, while initially scanning the data, I noticed that some had opted to use the "is_in" tag to denote a part of the city, furthermore pointing out a need for standardizing the data.

In [57]:
#Finds the number of entries under various tags with the "is_in" key

QUERY = '''
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='is_in'
GROUP BY value
ORDER BY num DESC
LIMIT 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'Santiago; Chile', 210),
 (u'Chile, Latin America', 128),
 (u'Lo Barnechea, Santiago, Regi\xf3n Metropolitana de Santiago, Chile', 11),
 (u'Las Condes, Santiago, Chile', 4),
 (u'Lo Barnechea; Santiago; Regi\xf3n Metropolitana de Santiago; Chile; Am\xe9rica del Sur',
  4),
 (u'Santiago, Chile', 4),
 (u'Huechuraba', 2),
 (u'Vitacura, Santiago', 2),
 (u'Huechuraba; Santiago; Regi\xf3n Metropolitana de Santiago; Chile; Am\xe9rica del Sur',
  1),
 (u'Las Condes, Santiago, Regi\xf3n Metropolitana de Santiago, Chile', 1)]


### Amenities

In [58]:
#Lists the top ten most common amenities
QUERY = '''
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'school', 1946),
 (u'restaurant', 1643),
 (u'kindergarten', 1053),
 (u'pharmacy', 749),
 (u'fast_food', 632),
 (u'parking', 586),
 (u'bank', 498),
 (u'cafe', 445),
 (u'bench', 306),
 (u'bicycle_parking', 241)]


In [59]:
# I tried to see if schools had any additional information, such as level of school, 
# In this data set, there is only "operator", which indicates who runs the school (government, private, church, etc.)
# though the results are clearly codified, I could not find what these values mean

QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='school') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='operator'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 20;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'3', 1241),
 (u'1', 261),
 (u'2', 178),
 (u'4', 154),
 (u'5', 26),
 (u'Ilustre Municipalidad de Maip\xfa', 1)]


In [60]:
#Values for the "highway" key from most to least common

QUERY = '''
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='highway' 
GROUP BY value
ORDER BY num DESC;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'bus_stop', 10865),
 (u'traffic_signals', 4020),
 (u'crossing', 1798),
 (u'turning_circle', 261),
 (u'give_way', 84),
 (u'motorway_junction', 61),
 (u'stop', 45),
 (u'mini_roundabout', 21),
 (u'elevator', 8),
 (u'rest_area', 8),
 (u'tunnel_entrance', 4),
 (u'traffic_signals;crossing', 1)]


In [61]:
#Values for the "railway" key from most to least common

QUERY = '''
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='railway' 
GROUP BY value
ORDER BY num DESC;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'subway_entrance', 294),
 (u'station', 143),
 (u'level_crossing', 35),
 (u'proposed', 11),
 (u'halt', 2)]


This is a little strange since infact there are only 108 metro stations and only two regional train stations... (Footnote #5)

In [62]:
# Top 20 types of restaurants in Santiago, not surprised..
# There is a lot of sushi here, but it's all filled with cream cheese :P

QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 20;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'sushi', 89),
 (u'chinese', 85),
 (u'pizza', 51),
 (u'regional', 29),
 (u'steak_house', 23),
 (u'italian', 21),
 (u'sandwich', 21),
 (u'peruvian', 20),
 (u'international', 16),
 (u'japanese', 15),
 (u'chicken', 10),
 (u'burger', 9),
 (u'seafood', 9),
 (u'Peruvian', 7),
 (u'indian', 6),
 (u'arab', 5),
 (u'asian', 5),
 (u'coffee_shop', 5),
 (u'spanish', 5),
 (u'mexican', 4)]


In [63]:
#Top ten data sources
QUERY = '''
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='source'
GROUP BY value
ORDER BY num DESC
LIMIT 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'http://www.transantiago.cl', 10683),
 (u'Bing', 4001),
 (u'Reconocimiento cartogr\xe1fico 2016 por KG.', 3415),
 (u'Reconocimiento cartogr\xe1fico de campo 2016 por KG.', 2369),
 (u'Gobierno Regional Metropolitano de Santiago (http://www.gobiernosantiago.cl/)',
  2001),
 (u'Reconocimiento cartogr\xe1fico de campo 2016 por KG', 1388),
 (u'KG Ground Survey 2016', 798),
 (u'GNS', 135),
 (u'Bicimapa - UAHC', 103),
 (u'Reconocimiento cartogr\xe1fico por KG 2016', 85)]


In [64]:
#Top ten amenities in Providencia

QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='Providencia') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='amenity'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'bicycle_parking', 104),
 (u'restaurant', 77),
 (u'cafe', 34),
 (u'kindergarten', 28),
 (u'pharmacy', 24),
 (u'fast_food', 18),
 (u'bank', 11),
 (u'pub', 11),
 (u'embassy', 7),
 (u'bureau_de_change', 6)]


### Amenities in Comunas

In [65]:
#Top ten comunas with bicycle parking

QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='bicycle_parking') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key LIKE '%city'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'Providencia', 104),
 (u'6', 37),
 (u'10', 19),
 (u'5', 19),
 (u'4', 14),
 (u'8', 13),
 (u'15', 9),
 (u'12', 8),
 (u'20', 7),
 (u'30', 6)]


In [66]:
#Top ten comunas by bus stop

QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='bus_stop') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key LIKE '%city'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'Puente Alto', 805),
 (u'Maip\xfa', 746),
 (u'La Florida', 669),
 (u'Santiago', 619),
 (u'San Bernardo', 476),
 (u'Pudahuel', 416),
 (u'Las Condes', 410),
 (u'\xd1u\xf1oa', 363),
 (u'La Pintana', 353),
 (u'Pe\xf1alol\xe9n', 353)]


In [67]:
#Top ten comunas by number of schools
QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='school') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key LIKE '%city'
GROUP BY nodes_tags.value
ORDER BY num DESC
Limit 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'Cerro Navia', 13),
 (u'Quinta Normal', 9),
 (u'Recoleta', 9),
 (u'Santiago', 7),
 (u'Independencia', 6),
 (u'Las Condes', 5),
 (u'Providencia', 5),
 (u'Pedro Aguirre Cerda', 4),
 (u'Buin', 3),
 (u'Estaci\xf3n Central', 3)]


Unfortunately, the other comunas have not been mentioned by name in the list of bicycle parking spots.  While Provi definitely does have the most bike parking in the city, it exists in the other areas as well. Additionally, these are not definitely not all the schools nor bus-stops in the city.

In [68]:
#Top ten amenities in Lo Barnechea
QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='Lo Barnechea') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='amenity'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'pharmacy', 10),
 (u'kindergarten', 9),
 (u'fast_food', 7),
 (u'bank', 5),
 (u'restaurant', 4),
 (u'school', 3),
 (u'bar', 1),
 (u'cafe', 1),
 (u'cinema', 1),
 (u'dentist', 1)]


Again, there are far more than three schools in LoBa.  I know, because I happen to work at one of them and can think of 5 others off the top of my head. :)

In [69]:
#Top ten comunas by number of restaurants
QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key LIKE '%city'
GROUP BY nodes_tags.value
ORDER BY num DESC
Limit 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'Providencia', 77),
 (u'Santiago', 42),
 (u'Las Condes', 33),
 (u'Vitacura', 21),
 (u'\xd1u\xf1oa', 19),
 (u'San Bernardo', 14),
 (u'Buin', 7),
 (u'La Reina', 7),
 (u'Macul', 5),
 (u'Lo Barnechea', 4)]


In [70]:
#Top ten comunas by number of banks
QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='bank') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key LIKE '%city'
GROUP BY nodes_tags.value
ORDER BY num DESC
Limit 10;'''

c.execute(QUERY)
rows = c.fetchall()

pprint.pprint(rows)

[(u'Las Condes', 27),
 (u'Santiago', 19),
 (u'Providencia', 11),
 (u'La Florida', 7),
 (u'San Bernardo', 7),
 (u'Vitacura', 6),
 (u'Lo Barnechea', 4),
 (u'La Reina', 2),
 (u'Buin', 1),
 (u'Quinta Normal', 1)]


This squares with what I know -- the financial hub of the city is in Las Condes, and Providencia has a lot of great eats. :)

## Summary and Ideas for Further Exploration

Though I was surprised by the amount of data present on the Santiago map, it is by no means complete.  Scraping from government websites as well as commerical sites like Tripadvisor would augment the information on schools and amenities and help to audit its consistency.  There does already seem to be a large amount of data already present on public transportation however.  This makes sense as the largest source of data is http://www.transantiago.cl.

Uniformity is a big issue for this data set, not just in terms of abbreviations for street names, but how items are tagged.  Sometimes street names are duplicated in multiple tags and many versions of the comunas (regions of the city) cloud the data.  Re-tagging the "city" and "is-in" for uniformity is one extension on the wrangling that has been done.

This case study attempted to make street abbreviations uniform, and change the keys for improper secondary tags (from addr:interpolation to addr:street) for example.  Additionally, querying was used to learn about the number of users, which amenities are present in the data, and where they can be found across the city.

Remaining questions include what the meaning of the "id_origin" tag is and what the numerical codes for the school operator tag mean.  In general, I wonder if there is a way to name these tags or include a file on OpenStreetMaps that makes these things more transparent.


### Footnotes

1. Project details -- cutting the file down to a sample: https://classroom.udacity.com/nanodegrees/nd002/parts/0021345404/modules/316820862075463/lessons/3168208620239847/concepts/77135319070923#
2. Changesets -- http://wiki.openstreetmap.org/wiki/Changeset
3. Stackoverflow on printing a dictionary sorted by values  http://stackoverflow.com/questions/11228812/print-a-dict-sorted-by-values
4. OSM highway wiki http://wiki.openstreetmap.org/wiki/Key:highway
5. Santiago Metro https://en.wikipedia.org/wiki/Santiago_Metro